# ITN Plots, DHS data

In [ ]:
# TODO These could be one large for-loop, through 'access' and 'use'

### Preliminary steps

In [ ]:
rm(list = ls())

options(scipen=999)

In [ ]:
# Global paths
Sys.setenv(PROJ_LIB = "/opt/conda/share/proj")
Sys.setenv(GDAL_DATA = "/opt/conda/share/gdal")

In [ ]:
# Paths
ROOT_PATH <- '~/workspace'
CONFIG_PATH <- file.path(ROOT_PATH, 'configuration')
CODE_PATH <- file.path(ROOT_PATH, 'code')
DATA_PATH <- file.path(ROOT_PATH, 'data')
DHS_DATA_PATH <- file.path(DATA_PATH, 'dhs', 'raw')
OUTPUT_DATA_PATH <- file.path(DATA_PATH, 'dhs', 'indicators', 'bednets')
OUTPUT_PLOTS_PATH <- file.path(ROOT_PATH, 'pipelines', 'snt_dhs_indicators', 'reporting', 'outputs')

In [ ]:
# Load utils
source(file.path(CODE_PATH, "snt_utils.r"))

# List required pcks
required_packages <- c("sf", "glue", "data.table", "ggplot2", "stringi", "jsonlite", "httr", "reticulate")

# Execute function
install_and_load(required_packages)

In [ ]:
Sys.setenv(RETICULATE_PYTHON = "/opt/conda/bin/python")
reticulate::py_config()$python
openhexa <- import("openhexa.sdk")

In [ ]:
# Load SNT config
CONFIG_FILE_NAME <- "SNT_config.json"
config_json <- tryCatch({ fromJSON(file.path(CONFIG_PATH, CONFIG_FILE_NAME)) },
                        error = function(e) {
                          msg <- paste0("Error while loading configuration", conditionMessage(e))  
                          cat(msg)   
                          stop(msg) 
                        })

msg <- paste0("SNT configuration loaded from  : ", file.path(CONFIG_PATH, CONFIG_FILE_NAME)) 
log_msg(msg)

# Set config variables
COUNTRY_CODE <- config_json$SNT_CONFIG$COUNTRY_CODE

dhis2_dataset <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_DATASET_FORMATTED

In [ ]:
# COUNTRY_CODE <- "BFA"
print(paste("Country code: ", COUNTRY_CODE))

In [ ]:
admin_level <- 'ADM1'
admin_id_col <- glue(admin_level, 'ID', .sep='_')
admin_name_col <- glue(admin_level, 'NAME', .sep='_')
admin_cols <- c(admin_id_col, admin_name_col)

## Geo data

In [ ]:
# Load spatial file from dataset
spatial_data_filename <- paste(COUNTRY_CODE, "shapes.geojson", sep = "_")
# spatial_data <- read_sf(file.path(DATA_PATH, 'dhis2', 'formatted', spatial_data_filename))
spatial_data <- get_latest_dataset_file_in_memory(dhis2_dataset, spatial_data_filename)
# log_msg(glue("File {spatial_data_filename} successfully loaded from dataset version: {dhis2_dataset}"))

In [ ]:
spatial_data <- st_as_sf(spatial_data)

# aggregate geometries by the admin columns
spatial_data <- aggregate_geometry(
  sf_data=spatial_data,
  admin_id_colname=admin_id_col,
  admin_name_colname=admin_name_col
)

# keep class
spatial_data <- st_as_sf(spatial_data)

if(COUNTRY_CODE == "COD"){
  spatial_data[[admin_name_col]] <- clean_admin_names(spatial_data[[admin_name_col]])
}

## Import DHS data

In [ ]:
data_source <- 'DHS'
indicator_access <- 'PCT_ITN_ACCESS'
indicator_use <- 'PCT_ITN_USE'

## ITN ACCCESS PLOTS

In [ ]:
filename_without_extension <- glue("{COUNTRY_CODE}_{data_source}_{admin_level}_{toupper(indicator_access)}")
bednet_access_table <- fread(file.path(OUTPUT_DATA_PATH, paste0(filename_without_extension, '.csv')))

In [ ]:
access_plot_data =  merge(spatial_data, bednet_access_table, by = admin_cols, all = TRUE)

In [ ]:
access_lower_bound_col <- glue("{toupper(indicator_access)}_CI_LOWER_BOUND")
access_upper_bound_col <- glue("{toupper(indicator_access)}_CI_UPPER_BOUND")
access_sample_avg_col <- glue("{toupper(indicator_access)}_SAMPLE_AVERAGE")

### ITN Access Map

In [ ]:
ITN_access_plot <- make_dhs_map(
      plot_dt = access_plot_data,
      plot_colname = access_sample_avg_col,
      title_name = "ITN Access (%)",
      legend_title = "%",
      scale_limits = c(0, 100)
    )

In [ ]:
ITN_access_plot_filename <- glue('{COUNTRY_CODE}_{data_source}_{admin_level}_{toupper(indicator_access)}_plot.png')
ggsave(ITN_access_plot, file = file.path(OUTPUT_PLOTS_PATH, ITN_access_plot_filename), dpi = 500)

In [ ]:
# confidence interval plot
access_ci_plot_title <- glue("{COUNTRY_CODE} {data_source} {toupper(indicator_access)} CI")
access_ci_plot_xlab <- admin_name_col
access_ci_plot_ylab <- glue("ITN access (%)")
access_ci_plot_filename <- glue("{COUNTRY_CODE}_{data_source}_{admin_level}_{toupper(indicator_access)}_CI_plot.png")

In [ ]:
access_ci_plot <- make_ci_plot(
  df_to_plot=access_plot_data,
  admin_colname=admin_name_col,
  point_estimation_colname=access_sample_avg_col,
  ci_lower_colname=access_lower_bound_col,
  ci_upper_colname=access_upper_bound_col,
  title_name=access_ci_plot_title,
  x_title=access_ci_plot_xlab,
  y_title=access_ci_plot_ylab
)

# ci_access_plot

In [ ]:
ggsave(filename=file.path(OUTPUT_PLOTS_PATH, access_ci_plot_filename), plot=access_ci_plot, width = 8, height = 6, dpi = 300)

## ITN USE PLOTS

In [ ]:
filename_without_extension <- glue("{COUNTRY_CODE}_{data_source}_{admin_level}_{toupper(indicator_use)}")
bednet_use_table <- fread(file.path(OUTPUT_DATA_PATH, paste0(filename_without_extension, '.csv')))

In [ ]:
use_plot_data =  merge(spatial_data, bednet_use_table, by = admin_cols, all = TRUE)

In [ ]:
use_lower_bound_col <- glue("{toupper(indicator_use)}_CI_LOWER_BOUND")
use_upper_bound_col <- glue("{toupper(indicator_use)}_CI_UPPER_BOUND")
use_sample_avg_col <- glue("{toupper(indicator_use)}_SAMPLE_AVERAGE")

### ITN use plot

In [ ]:
ITN_use_plot <- make_dhs_map(
      plot_dt = use_plot_data,
      plot_colname = use_sample_avg_col,
      title_name = "ITN Use (%)",
      legend_title = "%",
      scale_limits = c(0, 100)
    )

In [ ]:
ITN_use_plot_filename <- glue('{COUNTRY_CODE}_{data_source}_{admin_level}_{toupper(indicator_use)}_plot.png')
ggsave(ITN_use_plot, file = file.path(OUTPUT_PLOTS_PATH, ITN_use_plot_filename), dpi = 500)

In [ ]:
# confidence interval plot
use_ci_plot_title <- glue("{COUNTRY_CODE} {data_source} {toupper(indicator_use)} CI")
use_ci_plot_xlab <- admin_name_col
use_ci_plot_ylab <- glue("ITN Use (%)")
use_ci_plot_filename <- glue("{COUNTRY_CODE}_{data_source}_{admin_level}_{toupper(indicator_use)}_CI_plot.png")

In [ ]:
use_ci_plot <- make_ci_plot(
  df_to_plot=use_plot_data,
  admin_colname=admin_name_col,
  point_estimation_colname=use_sample_avg_col,
  ci_lower_colname=use_lower_bound_col,
  ci_upper_colname=use_upper_bound_col,
  title_name=use_ci_plot_title,
  x_title=use_ci_plot_xlab,
  y_title=use_ci_plot_ylab
)

In [ ]:
ggsave(filename=file.path(OUTPUT_PLOTS_PATH, use_ci_plot_filename), plot=use_ci_plot, width = 8, height = 6, dpi = 300)